In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import json
from pprint import pprint
import numpy as np
Settings = json.load(open('settings.txt'))
pprint(Settings)
import numpy as np
import sys
sys.path.append('../')
from cabbage.regression.Regression import get_W_mot16_02_dmax100
from cabbage.MultiplePeopleTracking import GraphGenerator
from cabbage.features.deepmatching import ReadOnlyDeepMatching
from cabbage.features.ReId import StoredReId, StackNet64x64, get_element
from experiments import MOT16_Experiments
from cabbage.data.video import VideoData
from time import time

print("\n")

dmax = 100

root = Settings['data_root']

mot16 = MOT16_Experiments(root)
video_name = 'MOT16-11'
X = mot16.mot16_11_X

Dt = mot16.mot16_11_detections
vd = VideoData(Dt)
Dt = vd.get_n_first_frames(100)

n, _ = Dt.shape
W, H = 64, 64

print("Dt", Dt.shape)
print("\n")

Im = np.zeros((n, H, W, 3), 'uint8')

IDS_IN_FRAME = [None] * (X.shape[0] + 1)
LAST_FRAME = X.shape[0]

for i, (frame, x, y, w, h, _) in enumerate(Dt):
    frame_i = int(frame)
    im = get_element(X[frame_i-1], (x,y,w,h), (W, H), True)
    Im[i] = im
    
    if IDS_IN_FRAME[frame_i] is None:
        IDS_IN_FRAME[frame_i] = []
    IDS_IN_FRAME[frame_i].append(i)


ALL_PAIRS = []
for frame_i, ids_in_frame in enumerate(IDS_IN_FRAME):
    if ids_in_frame is None:
        continue
    
    for i in ids_in_frame:
        for j in ids_in_frame:
            if i != j:
                ALL_PAIRS.append((i,j))
        
        for frame_j in range(frame_i + 1, min(frame_i + dmax + 1, LAST_FRAME)):
            if ALL_PAIRS[frame_j] is None:
                continue
            for j in ALL_PAIRS[frame_j]:
                ALL_PAIRS.append((i, j))
    
    print('handle frame ' + str(frame_i) + " from " + str(LAST_FRAME))
                
print(len(ALL_PAIRS))

{'data_root': '/home/data',
 'deepmatch': '/home/deepmatching_1.2.2_c++/deepmatching-static'}


Using TensorFlow backend.




Dt (3774, 6)


handle frame 1 from 900
handle frame 2 from 900
handle frame 3 from 900
handle frame 4 from 900
handle frame 5 from 900
handle frame 6 from 900
handle frame 7 from 900
handle frame 8 from 900
handle frame 9 from 900
handle frame 10 from 900
handle frame 11 from 900
handle frame 12 from 900
handle frame 13 from 900
handle frame 14 from 900
handle frame 15 from 900
handle frame 16 from 900
handle frame 17 from 900
handle frame 18 from 900
handle frame 19 from 900
handle frame 20 from 900
handle frame 21 from 900
handle frame 22 from 900
handle frame 23 from 900
handle frame 24 from 900
handle frame 25 from 900
handle frame 26 from 900
handle frame 27 from 900
handle frame 28 from 900
handle frame 29 from 900
handle frame 30 from 900
handle frame 31 from 900
handle frame 32 from 900
handle frame 33 from 900
handle frame 34 from 900
handle frame 35 from 900
handle frame 36 from 900
handle frame 37 from 900
handle frame 38 from 900
handle frame 39 from 900
handle frame 40 f

In [4]:
#reid.set_mot16_11_dmax100_true_predictions3349()
#reid = StackNet64x64(root)
reid = StoredReId(root, 100)

end = len(ALL_PAIRS)
for i in range(0, end, 500):
    PAIRS = np.array(ALL_PAIRS[i:i+500])
    a = PAIRS[:,0]
    b = PAIRS[:,1]
    A = Im[a]
    B = Im[b]
    X = np.concatenate([A, B], 3)    
    reid.batch_memorize(a, b, X)


reid.save('batch_mot16-11_dmax100')

Found model /home/data/reid_models/stacknet64x64_84_BOTH.h5! :)
attempt: 0:1
attempt: 0:2
attempt: 0:3
attempt: 0:4
attempt: 0:5
(5, 64, 64, 6)
